In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install geopandas
!pip install geopy

import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time

locator = Nominatim(user_agent= 'myGeocoder')

In [4]:
path = "/content/drive/My Drive/CNPJ/data/dados_cnpj_fortaleza_enderecos.csv"

data = pd.read_csv(path, encoding="ISO-8859-1")
print("Quantidade de dados: ", len(data))

data = data[['capital_social', 'cnae_fiscal', 'cnpj', 'cod_nat_juridica', 'matriz_filial', 'motivo_situacao', 'nome_fantasia', 'opc_simples', 'porte', 'qualif_resp', 'razao_social', 'situacao', 'telefone_1', 'telefone_2', 'data_inicio_ativ', 'data_situacao', 'email', 'endereco', 'lat', 'lon']]
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Quantidade de dados:  514576


,capital_social,cnae_fiscal,cnpj,cod_nat_juridica,matriz_filial,motivo_situacao,nome_fantasia,opc_simples,porte,qualif_resp,razao_social,situacao,telefone_1,telefone_2,data_inicio_ativ,data_situacao,email,endereco,lat,lon
0,0.0,9430800,508605031,3999,1,71,0,0,5,16,ASSOCIACAO DOS ROTARIANOS DO ROTARY CLUB FORTA...,8,0,0,19970225,20081231,0,"RUA SENADOR POMPEU, 1539, CENTRO, FORTALEZA, C...",-3,-38
1,200000.0,8610101,991604972,2240,1,1,0,0,1,49,AMH ASSISTENCIA MEDICO HOSPITALAR S/C LTDA,8,0,0,19970210,20190409,0,"RUA JOSE LOURENCO, 531, MEIRELES, FORTALEZA, C...",-3,-38
2,0.0,9430800,1679604964,3999,1,71,ASSOCIACAO SANTA EDWIRGES,0,5,16,ASSOCIACAO DE CONFECCIONISTAS DE FORTALEZA DO ...,8,0,0,19970409,20081231,0,"RUA VIA FERREA SOBRAL, 5381, ANTONIO BEZERRA, ...",-3,-38
3,0.0,6911701,1688604964,2232,1,0,0,0,5,49,ROCHA ARAUJO E ARRAIS ADVOGADOS ASSOCIADOS,2,0,0,19970418,20051103,0,"AVENIDA SANTOS DUMONT, 1687, ALDEOTA, FORTALEZ...",-3,-38
4,0.0,4615000,1790604981,2062,1,1,MOVEIS FLORENSE,0,1,49,FLORENSE NORDESTE SERVICOS E REPRESENTACOES LTDA,8,32641323,0,19970429,20091120,0,"RUA EDUARDO SALGADO, 443, ALDEOTA, FORTALEZA, ...",-3,-38


In [6]:
import numpy as np
convert_dict = dict(data.dtypes)
convert_dict['lat'] = np.float32
convert_dict['lon'] = np.float32

data = data.astype(convert_dict) 
data.dtypes

capital_social      float64
cnae_fiscal           int64
cnpj                  int64
cod_nat_juridica      int64
matriz_filial         int64
motivo_situacao       int64
nome_fantasia        object
opc_simples           int64
porte                 int64
qualif_resp           int64
razao_social         object
situacao              int64
telefone_1           object
telefone_2           object
data_inicio_ativ      int64
data_situacao         int64
email                object
endereco             object
lat                 float32
lon                 float32
dtype: object

### Obtendo coordenadas latitude e longitude dos endereços

In [0]:
# Instanciando geocoder
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

enderecos = data['endereco'].to_list() # lista dos endereços do dataset
fortaleza = geocode('Fortaleza') # pegando coordenadas de Fortaleza

In [0]:
import time
init = 0 
# data[data['lat'] == 0].index[0] # indíce da primeira ocorrência em que latitude é 0. Isso simboliza que a partir desse indíce os dados não foram mapeados
slots = [slot for slot in range(init, 500*((data.shape[0])//500), 500)] # lista de slots

for ind, slot in enumerate(slots): # Percorrendo os slots de dados
    print("Slot {} de {}".format(ind, len(slots)))
    teste = []
    for index, endereco in enumerate(enderecos[slot:slot+500]):
        try: # Tenta pegar a localização pelo endereço
            location = geocode(endereco)
            teste.append((location.latitude, location.longitude))
        except:
            try: # Caso não dê certo, pega a localização do Bairro em Fortaleza
                location = geocode(str(data['bairro'][index])+', Ceara')
                teste.append((location.latitude, location.longitude))
            except: # Caso não dê certo, pega a localização de Fortaleza
                location = fortaleza
                teste.append((location.latitude, location.longitude))

    for index, i in enumerate(list(range(slot,slot+500))): # Preenchimento dos dados no dataset original
        data['lat'][i] = teste[index][0]
        data['lon'][i] = teste[index][1]

    print("Escrevendo no .csv...")
    data.to_csv(path)
    print("Dando o tempo de 1 minuto")
    time.sleep(60)


Slot 0 de 1029


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Escrevendo no .csv...
Dando o tempo de 1 minuto
Slot 1 de 1029


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Escrevendo no .csv...
Dando o tempo de 1 minuto
Slot 2 de 1029
Escrevendo no .csv...
Dando o tempo de 1 minuto
Slot 3 de 1029
Escrevendo no .csv...
Dando o tempo de 1 minuto
Slot 4 de 1029
